In [17]:
import pandas as pd
import numpy as np
import re

## Demonstration

In [155]:
DATA_PATH = './ROS_Cellulo/ros_cellulo_practical/data/'

data_test = pd.read_csv(DATA_PATH+"Aggregation_field/tf_echo_00_06_66_D2_CF_7B-15-stdout.log", sep="\n")


In [165]:
def find_x(row):
    '''
    returns the first number of - Translation: [0.955, 0.605, 0.000]
    '''
    match = re.search('\[(.*)\,.*' , row)
    result = [x.strip() for x in match.group(1).split(',')]
    return result[0]

def find_y(row):
    '''
    returns the second number of - Translation: [0.955, 0.605, 0.000]
    '''
    match = re.search('\[(.*)\,.*' , row)
    result = [x.strip() for x in match.group(1).split(',')]
    return result[1]
#Ex of log:
    #Failure at 1554129972.459426873
    #Exception thrown:"00_06_66_D2_CF_7B" passed to lookupTransform argument source_frame does not exist. 
    #The current list of frames is:
    #Frame paper_world exists with parent base_footprint.

    #At time 1554129972.517
    #- Translation: [0.955, 0.605, 0.000]
    #- Rotation: in Quaternion [0.000, 0.000, -0.776, 0.630]
    #            in RPY (radian) [0.000, -0.000, -1.777]
    #            in RPY (degree) [0.000, -0.000, -101.840]
#Keeping only lines starting with "translation"
translation_data = data_test[data_test.squeeze().str.contains('Translation')].copy()

In [166]:
translation_data['x'] = translation_data.apply(lambda row: find_x(row.values[0]),axis=1)
translation_data['y'] = translation_data.apply(lambda row: find_y(row.values[0]),axis=1)

In [168]:
parsed_data = translation_data.drop(translation_data.columns[0], axis=1).\
                                drop(translation_data.index[0]).reset_index(drop=True)
parsed_data.head()

,x,y
0,65.176,448.452
1,65.136,448.442
2,66.533,448.889
3,66.406,448.737
4,66.289,448.727


## Real Work

In [188]:
situations = ["Aggregation_field", "Aggregation_field_param_lowMu", \
              "Aggregation_naive_neigh_obst","Aggregation_naive_obst_neigh", \
              "Coverage_10timesGainR", "Coverage_10timesGainR_low_Mu",\
              "Coverage_default", "Coverage_perturbation", "Interaction"]
robot_logs = ["tf_echo_00_06_66_D2_CF_7B-15-stdout.log",\
             "tf_echo_00_06_66_E7_8A_CA-14-stdout.log", \
             "tf_echo_00_06_66_E7_8A_D6-16-stdout.log", \
             "tf_echo_00_06_66_E7_8E_BD-17-stdout.log"]

robot_positions = []
#for all situation
for situation in situations:
    pos = []
    #for all robots
    for robot, robot_num in zip(robot_logs, range(4)):
        #import data
        df = pd.read_csv(DATA_PATH + situation + '/' + robot, sep="\n")
        #keeps only translation info
        translation_data = data_test[data_test.squeeze().str.contains('Translation')].copy()
        #parse to get x and y values
        translation_data['x'] = translation_data.apply(lambda row: find_x(row.values[0]),axis=1)
        translation_data['y'] = translation_data.apply(lambda row: find_y(row.values[0]),axis=1)
        
        pos.append(translation_data.drop(translation_data.columns[0], axis=1).\
                                drop(translation_data.index[0]).reset_index(drop=True))
    robot_positions.append(pos)
        
